사전순 정렬 대신 natural sort를 사용한다.

In [105]:
# cell 1

import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


먼저 파일을 읽어서 카테고리 이름을 요소로 갖는 string list를 만든다.

In [106]:
# cell 2

import os

path = '/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data'
folder_list = os.listdir(path)

category = []
# 카테고리 string 저장할 list

for i in folder_list:
    if not i.endswith('.DS_Store'):
    	# .DS_Store 빼고 추가
        category.append(i)
        
category.sort(key=natural_keys)

print(category)

['child', 'culture', 'economy', 'education', 'health', 'life', 'person', 'policy', 'society']


각 카테고리 안에 있는 파일 이름을 요소로 갖는 2D string list를 만든다.

In [107]:
# cell 3

fileName2D = []
# row: categoryIdx, column: 해당 카테고리의 파일 이름

for categoryIdx in range(len(category)):
    path = '/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data/'+category[categoryIdx]
    file_list = os.listdir(path)
    #해당 카테고리 안에 있는 파일명이 담긴 string list

    txt_list = []
    for i in file_list:
        if i.endswith('.txt') and not i.endswith('.DS_Store'):
            #파일 형식이 .txt로 끝나는 파일 이름만 배열에 추가
            txt_list.append(i)

    txt_list.sort(key=natural_keys)
    # natural sort
    fileName2D.append(txt_list)

print("카테고리마다의 파일 개수")
for i in fileName2D:
    print(len(i), end=" ")

# print("\n",fileName2D)


카테고리마다의 파일 개수
128 219 166 120 191 112 172 252 327 

### DTM 생성
모든 파일의 형태소를 중복 없이 요소로 갖는 DTM을 만든다.

DTM:

row: docNum, 0번 column: category,  나머지 column: term 인 2D Dataframe

빈 DataFrame을 생성했을 때 행과 열이 둘 다 없으면 데이터를 추가하거나 수정할 수 없다.

열 정보를 갖고 있어야 행 데이터를 추가할 수 있기 때문에 첫 column인 category를 추가해준다.

데이터 프레임 값 변경: df.loc[2, 'A'] = 3000

if morp == '':일때 컨티뉴 해주어야하는데 시간상  break로 첫 문단만 가져옴

In [108]:
# cell 4

import pandas as pd
import numpy as np

DTM = pd.DataFrame(columns=range(1))
DTM.columns = ['category']
# term table

docNum = 0
#문서 번호

for categoryIdx in range(len(category)):
    for fileIdx in range(len(fileName2D[categoryIdx])):
        file = open("/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data/"+category[categoryIdx]+"/"+fileName2D[categoryIdx][fileIdx], "r")

        lines = file.readlines()
        #한 줄씩 string으로 list에 저장

        docRow = "Doc"+str(docNum)
        #row 이름 ex) Doc3

        DTM.loc[docRow] = [0 for i in range(len(DTM.columns))]
        # ex)Doc3 = [0, 0, 0, 0](열 개수만큼 0을 만들어 list로 반환)

        DTM.loc[docRow, 'category'] = categoryIdx
        # 해당 문서의 카테고리 값에 categoryIdx 저장
        
        for line in lines:
            morp = line.split('\t')[-1].split('+')[0].strip('\n')
            #print(morp)
            # '\t'를 기준으로 뒤, '+'를 기준으로 앞의 string을 잘라 list에 저장
            if morp == '':
                break
########################################## continue로 바꿔야 함 ########################################
            # 빈 string이면 무시

            if morp in DTM.columns:
                DTM.loc[docRow,morp] += 1
            else:
                DTM[morp] = 0
                DTM.loc[docRow,morp] += 1
            # DTM columns에서 중복 확인, 중복 있으면 값만 1 증가시킴, 없으면 열 추가 후 값 1 대입 

        docNum += 1
                
DTM

### sum값 저장

(category column 에는 0 저장)

In [ ]:
from collections import Counter
# TT.loc['sum'] = TT.drop('category', axis=1).sum(axis=0)

DTM.loc['sum'] = DTM.sum(axis=0)
DTM.loc['sum','category'] = 0
DTM


,category,與/SL,성폭행/NNG,`/SW,거세/NNG,전면확대/NNG,추진/NNG,‘/SN,지원’/NNG,위하/VV,...,싸움/NNG,승소/NNG,스웨덴/NNP,43/SN,달러/NNB,국고보조금/NNG,농아인협회/NNG,광주인화학교/NNP,행정실장/NNG,감형/NNG
Doc0,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1683,8,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
Doc1684,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1685,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
Doc1686,8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


### DF값 저장
DTM을 .copy()로 깊은 복사한 DTM_df 사용

In [ ]:
# TT.loc['df'] = [0 for i in range(len(TT.columns))]
DTM_df = DTM.copy()

# TT dataframe column 개수
sum_remove = DTM_df.drop('sum')
# DTM에서 sum행 제거

df_list = []
# 새로 추가할 row인 'df'에 저장할 list

colNum = DTM.shape[1]
for c in range(colNum):
    df_list.append(len(sum_remove.loc[sum_remove.iloc[:,c] != 0]))
    # 모든 column을 다 돌면서 값이 0이 아닌 row가 몇개 있는지 df_list에 저장

DTM_df.loc['df'] = df_list
DTM_df.loc['df','category'] = 0

DTM_df

,category,與/SL,성폭행/NNG,`/SW,거세/NNG,전면확대/NNG,추진/NNG,‘/SN,지원’/NNG,위하/VV,...,싸움/NNG,승소/NNG,스웨덴/NNP,43/SN,달러/NNB,국고보조금/NNG,농아인협회/NNG,광주인화학교/NNP,행정실장/NNG,감형/NNG
Doc0,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1684,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1685,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
Doc1686,8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
sum,0,4,25,11,3,2,23,186,2,39,...,1,1,1,1,1,1,1,1,1,1


### 카테고리 label을 갖고 있는 (n,1) dataframe 생성

In [ ]:
label = pd.DataFrame(DTM_df['category'])
label

,category
Doc0,0
Doc1,0
Doc2,0
Doc3,0
Doc4,0
...,...
Doc1684,8
Doc1685,8
Doc1686,8
sum,0


### sum 값 기준으로 column 정렬
내림차순으로 정렬 후 1000번째 이후 요소는 삭제한다.

In [ ]:
DTM_df_sorted = DTM_df.drop('category', axis=1).sort_values(by='sum', axis= 1, ascending=False)
DTM_df_sorted = DTM_df_sorted.iloc[:,:1000]
# .loc은 row, column 이름으로, 
# .iloc은 row, column 인덱스로. 
DTM_df_sorted

,'/SS,‘/SL,‘/SN,장애인/NNG,"""/SS",“/SL,지원/NNG,위하/VV,내년/NNG,서울시/NNP,...,접근성/NNG,못하/VV,원전/NNG,죽음/NNG,농아인/NNG,보호견/NNG,걱정/NNG,기부/NNG,최대행사/NNG,한인/NNG
Doc0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1685,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1686,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sum,292,212,186,185,132,57,40,39,38,38,...,2,2,2,2,2,2,2,2,2,2


### IDF 함수 정의


In [ ]:
import math


def func_IDF(n, df):
    """
        args:
            n`int`: 문서의 총 개수
            df`int`: 행 하나의 df 값
        Retruns:
            idf`float`: 입력 받은 n, df 값으로 계산한 idf 값
    """
    return math.log(n / (1 + df))
    

idf_list = []

docNum = DTM_df_sorted.shape[0] - 2
print(docNum)
#sum, df row 제외한 순수 문서 개수

colNum = DTM_df_sorted.shape[1]
print(colNum)

for c in range(colNum):
    idf_list.append(func_IDF(docNum, DTM_df_sorted.loc['df'].iloc[c]))
    # 모든 column을 다 돌면서 값이 df 읽고 idf값 list에 추가

TF_IDF = DTM_df_sorted.iloc[:-3].copy()

# TF-IDF 테이블 생성
# for  돌려서 
docNum = TF_IDF.shape[0]
termNum = TF_IDF.shape[1]
for d in range(docNum) :
    for t in range(termNum):
        TF_IDF.iloc[d, t] = TF_IDF.iloc[d, t] * idf_list[t]
TF_IDF_std = (TF_IDF - TF_IDF.mean())/TF_IDF.std()
# 정규분포 
TF_IDF_std

1687
1000


,'/SS,‘/SL,‘/SN,장애인/NNG,"""/SS",“/SL,지원/NNG,위하/VV,내년/NNG,서울시/NNP,...,접근성/NNG,못하/VV,원전/NNG,죽음/NNG,농아인/NNG,보호견/NNG,걱정/NNG,기부/NNG,최대행사/NNG,한인/NNG
Doc0,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc1,-0.451961,-0.375106,2.741312,-0.350967,-0.291362,-0.187003,-0.155843,6.496593,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc2,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc3,2.157648,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc4,-0.451961,-0.375106,-0.339923,-0.350967,3.430123,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1681,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,-0.155843,6.496593,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc1682,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc1683,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,6.412924,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,29.008630,-0.034452,-0.034452
Doc1684,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452


### 벡터 추출해서 파일에 저장

In [ ]:
label.iloc[:-2, :]
TF_IDF_std

,'/SS,‘/SL,‘/SN,장애인/NNG,"""/SS",“/SL,지원/NNG,위하/VV,내년/NNG,서울시/NNP,...,접근성/NNG,못하/VV,원전/NNG,죽음/NNG,농아인/NNG,보호견/NNG,걱정/NNG,기부/NNG,최대행사/NNG,한인/NNG
Doc0,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc1,-0.451961,-0.375106,2.741312,-0.350967,-0.291362,-0.187003,-0.155843,6.496593,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc2,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc3,2.157648,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc4,-0.451961,-0.375106,-0.339923,-0.350967,3.430123,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1681,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,-0.155843,6.496593,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc1682,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452
Doc1683,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,6.412924,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,29.008630,-0.034452,-0.034452
Doc1684,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452


In [ ]:
TF_IDF_with_label = pd.concat([TF_IDF_std, label.iloc[:-2, :]],axis = 1)
TF_IDF_with_label



,'/SS,‘/SL,‘/SN,장애인/NNG,"""/SS",“/SL,지원/NNG,위하/VV,내년/NNG,서울시/NNP,...,못하/VV,원전/NNG,죽음/NNG,농아인/NNG,보호견/NNG,걱정/NNG,기부/NNG,최대행사/NNG,한인/NNG,category
Doc0,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,0
Doc1,-0.451961,-0.375106,2.741312,-0.350967,-0.291362,-0.187003,-0.155843,6.496593,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,0
Doc2,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,0
Doc3,2.157648,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,0
Doc4,-0.451961,-0.375106,-0.339923,-0.350967,3.430123,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1682,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,8
Doc1683,-0.451961,-0.375106,-0.339923,2.847578,-0.291362,-0.187003,6.412924,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,29.008630,-0.034452,-0.034452,8
Doc1684,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,8
Doc1685,-0.451961,-0.375106,-0.339923,-0.350967,-0.291362,-0.187003,-0.155843,-0.153836,-0.151804,-0.151804,...,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,-0.034452,8


In [ ]:
TF_IDF_with_label.to_excel('TF_IDF_with_label.xlsx')

In [ ]:

TF_IDF_with_label.to_csv("/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/TF-IDF/all_train_features.txt", sep = '\t', header= None, index=False)


### 평가 데이터 벡터로 준비
테스트 디렉토리에 있는 모든 파일 이름 lsit에 저장

In [ ]:
testFileName2D = []
# row: categoryIdx, column: 해당 카테고리의 파일 이름

for categoryIdx in range(len(category)):
    path = '/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Test_Data/'+category[categoryIdx]
    file_list = os.listdir(path)
    #해당 카테고리 안에 있는 파일명이 담긴 string list

    txt_list = []
    for i in file_list:
        if i.endswith('.txt') and not i.endswith('.DS_Store'):
            #파일 형식이 .txt로 끝나는 파일 이름만 배열에 추가
            txt_list.append(i)

    txt_list.sort(key=natural_keys)
    # natural sort
    testFileName2D.append(txt_list)

print("카테고리마다의 파일 개수")
for i in testFileName2D:
    print(len(i), end=" ")

print("\n",testFileName2D)

카테고리마다의 파일 개수
10 10 10 10 10 10 10 15 15 
 [['8_(POS)child_129.txt', '8_(POS)child_130.txt', '8_(POS)child_131.txt', '8_(POS)child_132.txt', '8_(POS)child_133.txt', '8_(POS)child_134.txt', '8_(POS)child_135.txt', '8_(POS)child_136.txt', '8_(POS)child_137.txt', '8_(POS)child_138.txt'], ['8_(POS)culture_220.txt', '8_(POS)culture_221.txt', '8_(POS)culture_222.txt', '8_(POS)culture_223.txt', '8_(POS)culture_224.txt', '8_(POS)culture_225.txt', '8_(POS)culture_226.txt', '8_(POS)culture_227.txt', '8_(POS)culture_228.txt', '8_(POS)culture_229.txt'], ['8_(POS)economy_167.txt', '8_(POS)economy_168.txt', '8_(POS)economy_169.txt', '8_(POS)economy_170.txt', '8_(POS)economy_171.txt', '8_(POS)economy_172.txt', '8_(POS)economy_173.txt', '8_(POS)economy_174.txt', '8_(POS)economy_175.txt', '8_(POS)economy_176.txt'], ['8_(POS)education_121.txt', '8_(POS)education_122.txt', '8_(POS)education_123.txt', '8_(POS)education_124.txt', '8_(POS)education_125.txt', '8_(POS)education_126.txt', '8_(POS)education_127

테스트 데이터 DTM 채우기

In [ ]:
TestDTM = pd.DataFrame(columns=range(TF_IDF_std.shape[1]))
TestDTM.columns = TF_IDF_std.columns
# term table

testdocNum = 0
#문서 번호

for categoryIdx in range(len(category)):
    for fileIdx in range(len(testFileName2D[categoryIdx])):
        file = open("/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Test_Data/"+category[categoryIdx]+"/"+testFileName2D[categoryIdx][fileIdx], "r")

        lines = file.readlines()
        #한 줄씩 string으로 list에 저장

        docRow = "Doc"+str(docNum)
        #row 이름 ex) Doc3

        TestDTM.loc[docRow] = [0 for i in range(len(TestDTM.columns))]
        # ex)Doc3 = [0, 0, 0, 0](열 개수만큼 0을 만들어 list로 반환)

        #TestDTM.loc[docRow, 'category'] = categoryIdx
        # 해당 문서의 카테고리 값에 categoryIdx 저장
        
        for line in lines:
            morp = line.split('\t')[-1].split('+')[0].strip('\n')
            #print(morp)
            # '\t'를 기준으로 뒤, '+'를 기준으로 앞의 string을 잘라 list에 저장
            if morp == '':
                continue
            # 빈 string이면 무시

            if morp in TestDTM.columns:
                TestDTM.loc[docRow,morp] += 1
            # DTM columns에서 중복 확인, 중복 있으면 값만 1 증가시킴, 없으면 열 추가 후 값 1 대입 

        docNum += 1
                
TestDTM

,'/SS,‘/SL,‘/SN,장애인/NNG,"""/SS",“/SL,지원/NNG,위하/VV,내년/NNG,서울시/NNP,...,접근성/NNG,못하/VV,원전/NNG,죽음/NNG,농아인/NNG,보호견/NNG,걱정/NNG,기부/NNG,최대행사/NNG,한인/NNG
Doc1986,0,2,2,1,0,5,0,6,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1987,6,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1988,0,0,1,0,0,0,4,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1989,7,0,3,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1990,0,2,2,0,1,1,9,4,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc2081,1,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc2082,1,11,10,7,0,0,1,8,1,0,...,0,0,0,2,0,0,0,0,0,0
Doc2083,0,0,0,0,0,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc2084,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


df, idf 계산 후 TF-IDF 생성

In [ ]:
# TT.loc['df'] = [0 for i in range(len(TT.columns))]

df_list = []
# 새로 추가할 row인 'df'에 저장할 list
idf_list = []
docNum = TestDTM.shape[0]
#문서 총 개수
colNum = TestDTM.shape[1]
#Term Index

for c in range(colNum):
    df_list.append(len(TestDTM.loc[TestDTM.iloc[:,c] != 0]))
# 모든 column을 다 돌면서 값이 0이 아닌 row가 몇개 있는지 df_list에 저장

for c in range(colNum):
    idf_list.append(func_IDF(docNum, df_list[c]))
# 모든 column을 다 돌면서 값이 df 읽고 idf값 list에 추가

for d in range(docNum) :
    for c in range(colNum):
        TestDTM.iloc[d, t] = TestDTM.iloc[d, t] * idf_list[t]

# TestTF_IDF = (TestDTM - TestDTM.mean())/TestDTM.std()
# # 정규분포 

# TestTF_IDF
TestDTM



ZeroDivisionError: float division by zero

TF-IDF 테이블 만들기

In [ ]:


for d in range(docNum) :
    for t in range(termNum):
        TF_IDF.iloc[d, t] = TF_IDF.iloc[d, t] * idf_list[t]
TF_IDF_std = (TF_IDF - TF_IDF.mean())/TF_IDF.std()
# 정규분포 
TF_IDF_std

98
1000


KeyboardInterrupt: 

### SVM 모델 import